In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from langchain_community.callbacks import get_openai_callback
from langchain_openai import ChatOpenAI

EMBEDDING_MODEL = 'aari1995/German_Semantic_STS_V2'
model_kwargs = {'device':'cpu'}
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the custom embedding model
embedding_model_wrapper = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/sanca/miniconda3/envs/urbia-rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name aari1995/German_Semantic_STS_V2. Creating a new one with MEAN pooling.


In [2]:
def print_matches(matches):
    for p in matches:
        print(f"\n\nMatch with similarity {p[1]}:\n{p[0].page_content}")

In [3]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.load_local("../res/forum_index", embedding_model_wrapper, allow_dangerous_deserialization=True)

In [4]:
# answer1 = vector_store.similarity_search_with_score("Kind wacht stündlich auf")
# print_matches(answer1)

In [5]:

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def get_rag_chain(vectorstore, llm):
    retriever = vectorstore.as_retriever()
    template = """Du bist ein hilfreicher Assistent für junge Eltern, der Informationen aus Foren sammelt, um Eltern bei der Beantwortung von Fragen über ihre Kinder zu helfen.

            Verwende die folgenden Konversationen aus Elternforen, um die Frage des Benutzers zu beantworten. Jede Zeile beginnt mit dem Namen des Benutzers, gefolgt von ":" und dann dem Kommentar, zum Beispiel so: "John: Bei mir ist es genauso."
            Verschiedene Konversationen können sich auf dasselbe Thema beziehen.
            <conversations>
            {context}
            </conversations>
            
            Wenn du die Unterhaltungen im Forum zitierst, gib bitte den Benutzernamen in deiner Antwort an. Hier ist ein Beispiel in <example> Tags:
            <example>
            Viele Nutzer sagen dass es normal ist. Cari234 sagt, z.B, dass sie täglich überfordert ist. Lomo2 hat gleiche Erfahrungen.
            </example>

            Schreib eine klare Antwort auf diese Frage:
            <question>
            {question}
            </question>

            Wenn du die Antwort nicht weißt, sag einfach, dass du es nicht weißt.
            Wenn du die Frage beantwortest, nenn bitte konkrete Beispiele und Tipps aus den Forendiskussionen und verallgemeinere die Details nicht.

            Bevor du antwortest, gehe die folgenden Schritte durch:
            1. Fasse jede Unterhaltung zusammen und extrahiere die relevanten Informationen, Details und Beispiele, die sich auf die Benutzerfrage beziehen. Gib die Zusammenfassungen in <summary>-Tags aus.
            2. Beantworte die Benutzerfrage auf der Grundlage der Zusammenfassungen von Schritt 1. Füge alle relevanten Informationen, Details und Beispiele ein. Behalte die Antwort auf maximal 5 Sätze. Gib diesen Text im <answer>-Tag aus.
            """

    prompt = ChatPromptTemplate.from_template(template)

    rag_chain = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
        | prompt
        | llm
        | StrOutputParser()
    )

    rag_chain_with_source = RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough()
        }
    ).assign(answer=rag_chain)
    return rag_chain_with_source

gpt4_preview = ChatOpenAI(model_name="gpt-4-1106-preview")
gpt4 = ChatOpenAI(model_name="gpt-4")
gpt3_5 = ChatOpenAI(model_name="gpt-3.5-turbo-0125")

opus = ChatAnthropic(model='claude-3-opus-20240229')
sonnet = ChatAnthropic(model='claude-3-sonnet-20240229')
haiku = ChatAnthropic(model='claude-3-haiku-20240307')


rag_opus = get_rag_chain(vector_store, opus)
rag_sonnet = get_rag_chain(vector_store, sonnet)
rag_haiku = get_rag_chain(vector_store, haiku)
rag_gpt4_preview = get_rag_chain(vector_store, gpt4_preview)
rag_gpt4 = get_rag_chain(vector_store, gpt4)
rag_gpt3_5 = get_rag_chain(vector_store, gpt3_5)

In [6]:
# with get_openai_callback() as cb:
#     answer = rag_gpt3_5.invoke("Kind wird nachts stündlich wach")
#     print(cb)
# print(answer['answer'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Tokens Used: 2581
	Prompt Tokens: 2214
	Completion Tokens: 367
Successful Requests: 1
Total Cost (USD): $0.0016575
<summary>
SteMatz berichtet, dass ihr Baby alle 1-2 Stunden unruhig aufwacht, was zu Schlafmangel und Zerstreutheit führt. Muriel erklärt, dass die Schlafphase, in der man geweckt wird, auch wichtig ist. lattemachiatto erwähnt, dass die Schlafumgebung und Mängel wie Vitamin D oder Eisenmangel den Schlaf beeinflussen können. Mami823 teilt ihre Erfahrungen mit ihrem Baby, das alle 2 Stunden zum Schlafen gebracht werden musste. Michelleebwn berichtet von ihrem Baby, das nachts weinend aufwacht, möglicherweise aufgrund von Zähnen oder neuen Fähigkeiten wie dem Robben. Wunschmama2o20 erwähnt, dass solche Phasen nach Wachstumsschüben vorübergehen können. anonym676 beschreibt, wie ihr 4 Monate altes Baby stündlich aufwacht, viel trinkt und nur in ihrem Arm schläft, was zu Müdigkeit und Sorgen führt.
</summary>
<answer>
Es scheint, dass die häufigen nächtlichen Unterbrechungen bei

In [9]:
# with get_openai_callback() as cb:
#     answer = rag_gpt4_preview.invoke("Kind wird nachts stündlich wach")
#     print(cb)
# print(answer['answer'])

Tokens Used: 2805
	Prompt Tokens: 2214
	Completion Tokens: 591
Successful Requests: 1
Total Cost (USD): $0.03987
<summary>
SteMatz spricht über unruhigen Schlaf ihres Kindes und dass es etwa alle 1-2 Stunden aufwacht. Sie gewöhnt sich an den Schlafmangel, auch wenn es schwierig ist. Blabliblu meint, man gewöhne sich an regelmäßiges nächtliches Aufwachen und dass es einen Unterschied macht, ob man selbst aufwacht oder geweckt wird. Muriel weist darauf hin, dass die Schlafphase, in der man geweckt wird, und die eigene Gesundheit eine Rolle spielen. lattemachiatto führt Müdigkeit auf Vitamin D- und Eisenmangel zurück und sagt, dass das Baby in ihrer Nähe besser schlafen würde. Mami823 berichtet, dass ihr Baby bei regelmäßigem Schlafversuch alle 2 Stunden weniger übermüdet war und empfiehlt, durchzuhalten, da die Entwicklung des Babys solche Phasen mit sich bringt. Michelleebwn berichtet, dass ihr 9 Monate altes Baby nachts oft weinend aufsteht, obwohl es normalerweise gut weiterschläft, n

In [10]:
# with get_openai_callback() as cb:
#     answer = rag_gpt4.invoke("Kind wird nachts stündlich wach")
#     print(cb)
# print(answer['answer'])

Tokens Used: 2689
	Prompt Tokens: 2214
	Completion Tokens: 475
Successful Requests: 1
Total Cost (USD): $0.09491999999999999
<summary>
SteMatz, Blabliblu und Muriel diskutieren über die Schwierigkeiten, die sie mit ihren Babys haben, die nachts aufwachen. Sie betonen, dass es normal ist, dass Babys unruhig schlafen und dass Eltern lernen müssen, mit Schlafentzug umzugehen. Sie betonen auch, dass das Aufwachen in der falschen Schlafphase zu erhöhter Müdigkeit führen kann. Lattemachiatto weist darauf hin, dass ein Vitamin D- oder Eisenmangel dazu führen kann, dass man sich müder fühlt. Mami823 erwähnt, dass ihr Baby nachts nie ein Bäuerchen gemacht hat und dass es direkt in ihrem Beistellbett gefüttert wurde. 
Michelleebwn berichtet, dass ihr 9 Monate altes Baby nachts weinend aufwacht und fragt nach Lösungsvorschlägen. Wunschmama2o20 antwortet, dass ihre Tochter ähnliches Verhalten zeigte, wenn sie einen Wachstumsschub hatte und etwas Neues gelernt hat.
Anonym676 beschreibt eine ähnlich

In [11]:
# answer = rag_haiku.invoke("Kind wird nachts stündlich wach")
# print(answer['answer'])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Gut, lass uns die Forenbeiträge analysieren und eine Antwort für dich finden.

<summary>
SteMatz berichtet, dass ihr Baby nachts unruhig schläft und sie alle 1-2 Stunden wach wird. Blabliblu erklärt, dass man sich mit der Zeit daran gewöhnt, wenn die Blutwerte passen. Muriel fügt hinzu, dass es darauf ankommt, in welcher Schlafphase das Baby geweckt wird, und dass Hormonschwankungen nach der Geburt eine Rolle spielen können. lattemachiatto nennt mögliche Gründe wie Vitamin-D-Mangel, Eisenmangel oder wenn das Baby zu weit entfernt schläft, so dass die Schlafphasen nicht synchron sind.
</summary>

<summary>
Mami823 berichtet, dass ihr Baby mit ca. 3,5 Monaten sehr unruhig wurde, nur noch kurze Schlafphasen hatte und ständig quengelig war. Sie musste das Baby alle 2 Stunden zum Schlafen bringen. Sie erklärt, dass solche Phasen normal sind und es immer wieder Herausforderungen geben wird, zum Beispiel beim Zahnen. Wichtig sei, durchzuhalten und nicht zu denken, das Baby hasse einen.
</summ

In [ ]:
# answer = rag_sonnet.invoke("Kind wird nachts stündlich wach")
# print(answer['answer'])

In [ ]:
# answer = rag_opus.invoke("Kind wird nachts stündlich wach")
# print(answer['answer'])

In [1]:
import streamlit as st

def generate_response(query_text):
    answer = rag_haiku.invoke(query_text)
    return answer['answer']

# Page title
st.set_page_config(page_title='👶🏼 Kinder Forum')
st.title('👶🏼 Kinder Themen 👶🏼')

# Query text
query_text = st.text_input('Worum geht es?', placeholder = 'Baby schreit nachts.')

# Form input and query
result = []
with st.form('myform', clear_on_submit=True):
    submitted = st.form_submit_button('Submit', disabled=not(query_text))
    if submitted:
        with st.spinner('Suchen...'):
            response = generate_response(query_text)
            result.append(response)
if len(result):
    st.info(response)


2024-04-08 15:05:08.131 
  command:

    streamlit run /Users/sanca/miniconda3/envs/urbia-rag/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
